In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics.scorer import make_scorer
import xgboost as xgb
from matplotlib import pyplot as plt

import sys
sys.path.append('..')
from utils import preprocess, missing, evaluate

### Globals and load data

In [59]:
target = 'SI.POV.DDAY'
predict_year=2010
#percent of input Indicators to use (set to 100 for full set of input features)
percent = 25

#Load the data from disk
input_dir = '.\\..\\data\\'
data_input = "cleaned_data.pkl"
data = pd.read_pickle(input_dir + data_input)

#Possible subset of data choosen to reduce calulation time
#For percetages less than 100% we try to choose a subset that represents the spread of variables

if percent == 100:
    pass
else: 
    num_indicators_original = data.shape[1]
    step = int(100/percent)
    data_new = data.iloc[:,::step].copy()
    #Add the target column if not already included
    if target not in data_new.columns:
        data_new[target] = data[target]
    data = data_new
    
print(data.shape[1], "indicators included")

206 indicators included


### Window data and preprocess

In [60]:
%time data_regressors, data_targets = \
        preprocess.window_data(data, lag=3,num_windows=10, step=1, predict_year=2010, \
                         target=target, impute_type='interpolation')

#Break up into training and testing data.

idx = pd.IndexSlice
data_train_regressors = data_regressors.loc[idx[:,2:10],:]
data_train_targets = data_targets.loc[idx[:,2:10],:]
data_test_regressors = data_regressors.loc[idx[:,1],:]
data_test_targets= data_targets.loc[idx[:,1],:]

#For Training, only consider windows that don't have a missing target as they offer nothing to training
#Therefore, remove those observations from both the training regressors and targets datasets.
data_train_regressors_subset = data_train_regressors[~np.isnan(list(data_train_targets.values.flatten()))]
data_train_targets_subset = data_train_targets[~np.isnan(list(data_train_targets.values.flatten()))]

#For testing, also remove windows with no target variable as it is impossible to measure preformance.
data_test_regressors_subset = data_test_regressors[~np.isnan(list(data_test_targets.values.flatten()))]
data_test_targets_subset = data_test_targets[~np.isnan(list(data_test_targets.values.flatten()))]

Wall time: 1min 49s


In [5]:
X_train = data_train_regressors_subset.values
y_train = data_train_targets_subset.values.ravel()
X_test = data_test_regressors_subset.values
y_test = data_test_targets_subset

### XGBoost Model

#### Out-of-the-box using the Scikit-learn interface

In [6]:
XGB = xgb.XGBRegressor(random_state=42)
XGB.fit( X_train,y_train)
#Make predictions
predictions = XGB.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("MSE of XGBoost out-of-the-box is:", mse)

[13:47:15] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE of XGBoost out-of-the-box is: 28.249442133560215


#### Tuning of the Algorithm

In [42]:
cv_folds = 5

scorer = make_scorer(mean_squared_error ,greater_is_better=False)

Step 1. Tune the number of estimators

In [48]:
model = xgb.XGBRegressor(random_state=42, 
                       max_depth=5, 
                       min_child_weight = 1, 
                       gamma = 0, 
                       subsample=0.8, 
                       colsample_bytree = 0.8, 
                       scale_pos_weight = 1)

param = model.get_xgb_params()
data_matrix = xgb.DMatrix(X_train, label=y_train)
cvresult = xgb.cv(param, data_matrix, num_boost_round=model.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=50)
#Set the optimised number of estimators
model.set_params(n_estimators=cvresult.shape[0])

[14:54:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:54:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.8, verbosity=1)

In [50]:
best_so_far_model = model
best_so_far_model.fit( X_train,y_train)
#Make predictions
predictions = best_so_far_model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("MSE of xgboost after tuning (step 1) is:", mse)

[14:56:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE of xgboost after tuning (step 1) is: 25.366613176210667


Step 2. Tune max_depth and min_child_weight

In [52]:
params = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

model = xgb.XGBRegressor( learning_rate =0.1, n_estimators=100, max_depth=5,
                           min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                           objective='reg:squarederror', nthread=4, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None,
       subsample=0.8, verbosity=1),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [53]:
best_so_far_model = gsearch1.best_estimator_
best_so_far_model.fit( X_train,y_train)
#Make predictions
predictions = best_so_far_model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("MSE of xgboost after tuning (step 2) is:", mse)

MSE of xgboost after tuning (step 2) is: 24.77861493310224


Step 3. Tune Gamma

In [54]:
params = {
 'gamma':[i/10.0 for i in range(0,5)]
}

model = gsearch1.best_estimator_

gsearch3 = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None,
       subsample=0.8, verbosity=1),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [56]:
gsearch3.best_params_

{'gamma': 0.0}

In [55]:
best_so_far_model = gsearch3.best_estimator_
best_so_far_model.fit( X_train,y_train)
#Make predictions
predictions = best_so_far_model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("MSE of xgboost after tuning (step 3) is:", mse)

MSE of xgboost after tuning (step 3) is: 24.77861493310224


Step 4. Tune Regularization

In [57]:
params = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1]
}

model = gsearch3.best_estimator_

grid_step4 = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5 ,return_train_score=True)
grid_step4.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0.0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None,
       subsample=0.8, verbosity=1),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [1e-05, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [58]:
grid_step4.best_params_

{'reg_alpha': 0.1}

In [46]:
grid_step4.cv_results_

{'mean_fit_time': array([ 9.40506611,  9.37564592,  9.82336283, 10.1239759 ]),
 'std_fit_time': array([0.1205106 , 0.07282836, 0.37370569, 0.29931526]),
 'mean_score_time': array([0.01761298, 0.01501026, 0.02121539, 0.01681185]),
 'std_score_time': array([0.00417924, 0.00384914, 0.00462477, 0.00487812]),
 'param_reg_alpha': masked_array(data=[1e-05, 0.01, 0.1, 1],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'reg_alpha': 1e-05},
  {'reg_alpha': 0.01},
  {'reg_alpha': 0.1},
  {'reg_alpha': 1}],
 'split0_test_score': array([-6.40738403, -6.38016922, -6.64283264, -6.46345512]),
 'split1_test_score': array([-17.27572003, -17.49499098, -17.67286162, -16.86207948]),
 'split2_test_score': array([-34.68918529, -32.71073454, -34.75891266, -31.9309044 ]),
 'split3_test_score': array([-11.62166216, -10.32274597, -10.60072565,  -9.73275872]),
 'split4_test_score': array([-19.1275803 , -18.37638463, -17.74234072, -16.68877948]),
 

In [59]:
best_so_far_model = grid_step4.best_estimator_
best_so_far_model.fit( X_train,y_train)
#Make predictions
predictions = best_so_far_model.predict(X_test) 

mse= mean_squared_error(y_test, predictions)
print("MSE of xgboost after tuning (step 4) is:", mse)

MSE of xgboost after tuning (step 4) is: 24.639867157317262


### XGBoost Model with no Imputation

In [62]:
cv_folds = 5

scorer = make_scorer(mean_squared_error ,greater_is_better=False)

In [63]:
%time data_regressors, data_targets = \
        preprocess.window_data(data, lag=3,num_windows=10, step=1, predict_year=2010, \
                         target=target)


Wall time: 1.54 s


In [64]:
#Break up into training and testing data.

idx = pd.IndexSlice
data_train_regressors = data_regressors.loc[idx[:,2:10],:]
data_train_targets = data_targets.loc[idx[:,2:10],:]
data_test_regressors = data_regressors.loc[idx[:,1],:]
data_test_targets= data_targets.loc[idx[:,1],:]

In [65]:
#For Training, only consider windows that don't have a missing target as they offer nothing to training
#Therefore, remove those observations from both the training regressors and targets datasets.
data_train_regressors_subset = data_train_regressors[~np.isnan(list(data_train_targets.values.flatten()))]
data_train_targets_subset = data_train_targets[~np.isnan(list(data_train_targets.values.flatten()))]

#For testing, also remove windows with no target variable as it is impossible to measure preformance.
data_test_regressors_subset = data_test_regressors[~np.isnan(list(data_test_targets.values.flatten()))]
data_test_targets_subset = data_test_targets[~np.isnan(list(data_test_targets.values.flatten()))]

In [66]:
X_train_miss = data_train_regressors_subset.values
y_train_miss = data_train_targets_subset.values.ravel()
X_test_miss = data_test_regressors_subset.values
y_test_miss = data_test_targets_subset

#### Out-of-the-box xgboost on data without imputation

In [67]:
XGB = xgb.XGBRegressor(random_state=42, n_estimators=100,  objective='reg:squarederror',max_depth=2)
XGB.fit( X_train_miss,y_train_miss)
#Make predictions
predictions = XGB.predict(X_test_miss) 

mse= mean_squared_error(y_test_miss, predictions)
print("MSE of XGBoost out-of-the-box is:", mse)

MSE of XGBoost out-of-the-box is: 22.089705414468636


In [68]:
XGB.score(X_train_miss, y_train_miss)

0.9928193439050952

Step 2. Tune max_depth and min_child_weight

In [10]:
params = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

model = xgb.XGBRegressor( learning_rate =0.1, n_estimators=100, max_depth=5,
                           min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                           objective='reg:squarederror', nthread=4, scale_pos_weight=1, seed=27)

grid_step2_missing = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=cv_folds)
grid_step2_missing.fit(X_train_miss,y_train_miss)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=4, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=None,
       subsample=0.8, verbosity=1),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [13]:
best_so_far_model = grid_step2_missing.best_estimator_
best_so_far_model.fit( X_train_miss,y_train_miss)
#Make predictions
predictions = best_so_far_model.predict(X_test_miss) 

mse= mean_squared_error(y_test_miss, predictions)
print("MSE of xgboost after tuning (step 2) is:", mse)

MSE of xgboost after tuning (step 2) is: 22.470759551217924


In [22]:
params = {
 'reg_alpha':[0, 1e-5, 1e-2, 0.1, 1]
}

model =  xgb.XGBRegressor(random_state=42, n_estimators=100,  objective='reg:squarederror',max_depth='3')

grid_step4 = GridSearchCV(model, param_grid = params, scoring=scorer,
                        n_jobs=4,iid=False, cv=5 ,return_train_score=True)
grid_step4.fit(X_train_miss,y_train_miss)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth='3', min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'reg_alpha': [0, 1e-05, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(mean_squared_error, greater_is_better=False),
       verbose=0)

In [23]:
grid_step4.best_params_

{'reg_alpha': 0.01}

In [24]:
grid_step4.cv_results_

{'mean_fit_time': array([12.46490774, 13.33110442, 13.2894845 , 14.97988238, 13.31553102]),
 'std_fit_time': array([0.72104281, 0.68773044, 0.66761424, 0.80985538, 3.23645039]),
 'mean_score_time': array([0.01417837, 0.01425643, 0.01100802, 0.01141543, 0.01285686]),
 'std_score_time': array([0.00217707, 0.00973255, 0.00109672, 0.00205277, 0.00235153]),
 'param_reg_alpha': masked_array(data=[0, 1e-05, 0.01, 0.1, 1],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'reg_alpha': 0},
  {'reg_alpha': 1e-05},
  {'reg_alpha': 0.01},
  {'reg_alpha': 0.1},
  {'reg_alpha': 1}],
 'split0_test_score': array([-13.63875802, -13.63875679, -13.91272041, -15.06636568,
        -15.41895681]),
 'split1_test_score': array([-24.80837647, -24.8083728 , -21.9811472 , -21.98904935,
        -21.91256844]),
 'split2_test_score': array([-50.2840616 , -50.28405045, -51.14627544, -50.97711774,
        -48.56329881]),
 'split3_test_score': arra

In [25]:
best_so_far_model = grid_step2_missing.best_estimator_
best_so_far_model.fit( X_train_miss,y_train_miss)
#Make predictions
predictions = best_so_far_model.predict(X_test_miss) 

mse= mean_squared_error(y_test_miss, predictions)
print("MSE of xgboost after tuning (step 2) is:", mse)

MSE of xgboost after tuning (step 2) is: 22.470759551217924
